In [1]:
#Load packages
import numpy as np
from numpy.linalg import matrix_rank
import scipy.io as sio
import matplotlib.pyplot as plt
import random
from scipy.stats import multivariate_t
import scipy
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

In [2]:
#functions
def cal_t_params(xtrain, ytrain, Sigma0, kappa, m, mu0):
    all_loc_vec = [] # each has shape dx1
    all_scale_mat = [] # each has shape dxd
    all_dof = [] # each has shape 1
    d = xtrain.shape[1]
    all_labels = np.unique(ytrain)
    for label in all_labels:
        xtrain_sub = xtrain[ytrain==label]
        n = xtrain_sub.shape[0]
        xpar = np.squeeze(np.mean(xtrain_sub, axis=0))
        all_loc_vec.append((kappa*mu0 + n*xpar/(n+kappa)))
        diff = mu0-xpar
        term2 = Sigma0 + (n-1)*np.cov(xtrain_sub, rowvar=False) + (n*kappa)/(n+kappa)* np.outer(diff, diff)                 
        term2 *= (n + kappa+1)/((n+kappa)*(n+m+1-d))
        all_scale_mat.append(term2)
        all_dof.append(n+m+1-d)
    
    return all_loc_vec, all_scale_mat, all_dof

def predict(xtest, all_loc_vec, all_scale_mat, all_dof, all_labels):
    nr_classes = len(all_labels)
    log_lik = np.zeros((xtest.shape[0], nr_classes))
    for i in range(nr_classes):
        log_lik[:, i] = multivariate_t.logpdf(xtest, all_loc_vec[i], all_scale_mat[i], all_dof[i])
                                             
    predicted_class_ids = np.argmax(log_lik, axis=1)
    ypred = np.zeros(len(xtest))
    
    for i in range(len(predicted_class_ids)):
        ypred[i] = all_labels[predicted_class_ids[i]]
    
    return ypred

def splitData(data_path):
    mat = scipy.io.loadmat(data_path)

    tri = np.where((mat['pixims']>=1) & (mat['pixims']<= 55))[0]
    tsi = np.where((mat['pixims']>=56) & (mat['pixims'] <=77))[0]

    xtrain = mat['pixspec'][tri,0:248]
    ytrain = np.squeeze(mat['pixlabs'][tri,:])

    xtest = mat['pixspec'][tsi,0:248]
    ytest = np.squeeze(mat['pixlabs'][tsi,:])
    print(xtrain.shape, ytrain.shape, xtest.shape, ytest.shape)

    all_labels = np.unique(ytrain)
    
    return xtrain, ytrain, xtest, ytest, all_labels

def cal_prior(xtrain, ytrain, m):
    """
    Estimate prior information.
    """
    d = xtrain.shape[1]
    uy = np.unique(ytrain)
    nc = len(uy)
    mu0 = np.zeros((1,d))
    Sigma0 = np.zeros((d,d))
    for i in range(nc):
        idx = (ytrain==uy[i]).flatten()
        if len(xtrain[idx,:]) == 1:
            mu0 += xtrain[idx,:] 
        else:
            mu0 = mu0 + np.mean(xtrain[idx,:], axis=0)
            Sigma0 = Sigma0 + np.cov(xtrain[idx,:].T)
    mu0 = mu0/nc
    Sigma0 = (m-d-1)*Sigma0/nc
    
    return np.squeeze(mu0), Sigma0

def getSampleMeanCovariance(X_train, y_train, k, d, classes):
    x_bar = np.zeros((k,d))
    S = np.zeros((k,d,d))
    
    for i, kls in enumerate(classes):
        data_i = X_train[y_train == kls, :]
        x_bar[i,:] = np.mean(data_i, axis=0)
        S[i,:] = np.cov(data_i, rowvar = False)
    
    return x_bar, S

def report_mean_class_acc(ytest, ypred, all_labels):
    class_accuracies = []
    for label in all_labels:
        if(np.sum(ytest==label)>0):
            acc = np.sum(np.logical_and(ytest == label,ypred == label))/np.sum(ytest==label)
            class_accuracies.append(acc)
    mean_acc = np.mean(class_accuracies)
        
    return mean_acc

#This function is used to encode labels since labels are categorical.
def encode_labels(labels):
    le = LabelEncoder()
    le.fit(labels)
    encoded_labels = le.transform(labels)
    
    return encoded_labels, le

def decode_labels(encoded_predict_labels, le):
    test_predictions = le.inverse_transform(encoded_predict_labels)
    
    return test_predictions

In [10]:
#Load data
train = sio.loadmat('train.mat')
validation = sio.loadmat('validation.mat')
test = sio.loadmat('test_wolabels.mat')

train_classid = np.squeeze(train['classid'])
train_class_labels = []
for item in train_classid:
    train_class_labels.append(item[0])
train_features = train['features']
train_imid = train['imid']
train_sampleid = train['sampleid']
print(train_features.shape)
train_unique_labels_count = len(train_unique_labels)
print(train_unique_labels_count)

validation_classid = np.squeeze(validation['classid'])
validation_class_labels = []
for item in validation_classid:
    validation_class_labels.append(item[0])
validation_features = validation['features']
validation_imid = validation['imid']
validation_sampleid = validation['sampleid']
print(validation_features.shape)
validation_unique_labels = sorted(np.unique(validation_class_labels))
validation_unique_labels_count = len(validation_unique_labels)
print(validation_unique_labels_count)

#encoded train labels
train_labels, le = encode_labels(train_class_labels)
train_unique_labels = sorted(np.unique(train_labels))
print(len(train_labels))

#encoded validation labels
validation_labels = le.transform(validation_class_labels)
print(len(validation_labels))

(7849, 2048)
1013
(1379, 2048)
1013
7849
1379


In [11]:
print(train_unique_labels)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [5]:
#Apply PCA
pca = PCA(n_components=500)
train_features_red = pca.fit_transform(train_features)
print("Projection of train datapoints shape: ", train_features_red.shape)
validation_features_red = pca.transform(validation_features)
print("Projection of validation datapoints shape: ", validation_features_red.shape)

Projection of train datapoints shape:  (7849, 500)
Projection of validation datapoints shape:  (1379, 500)


In [6]:
#0-1 normalization
from sklearn.preprocessing import MinMaxScaler

scalar = MinMaxScaler()
train_features_norm = scalar.fit_transform(train_features_red)
validation_features_norm = scalar.transform(validation_features_red)

In [12]:
# tune kappa and m for best accuracy

all_configs = []
all_accs = []
best_acc = 0

d = train_features_red.shape[1]

all_kappas = [0.1, 1.0, 10.0]
all_ms = [d+2, 2*d, 10*d, 100*d, 1e3*d, 1e5*d, 1e8*d, 1e10*d]

for kappa in all_kappas:
    for m in all_ms:
        
        mu0, Sigma0 = cal_prior(train_features_red, train_labels, m)
        
        all_loc_vec, all_scale_mat, all_dof = cal_t_params(train_features_norm, train_labels, Sigma0, kappa, m, mu0)           
        ypred = predict(validation_features_norm, all_loc_vec, all_scale_mat, all_dof, train_unique_labels)
        acc= report_mean_class_acc(validation_labels, ypred, train_unique_labels)
        
        all_configs.append((kappa, m))
        all_accs.append(acc)
        
        if acc > best_acc:
            out_str = "kappa=%.3f \t m=%d" % (kappa, m)
            out_str += "\tfind better acc %.3f" %(acc)
            print(out_str)
            best_acc = acc

kappa=0.100 	 m=502	find better acc 0.705
